In [44]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
from datetime import datetime, timedelta
import time
import random
from modules.peers_view import PeersView
from modules.user import User
from modules.network import Network
from modules import config
import numpy as np

# Building routes

We consider 3-layers routes:

    A - L1 - RV - L2 - B

Such that RV and L2 are chosen by B, while L1 is chosen by A.

A layer consists of several candidate nodes to route a packet: only one of these candidates will route a packet at once.

We fill layers incrementally: while not satisfied, add another node to one of the layers.

Our problem is to find a stopping criterion.

$o_d$: probability for device $d$ to be online

$$ P[A \rightarrow B] \propto \sum\limits_{r \in L1}o_r * \sum\limits_{r \in RV}o_r * \sum\limits_{r \in L2}o_r $$ 

## Perform experiment

In [45]:
def init_experiment(conf=config.default):
    address_book = PeersView(conf)
    net = Network()

    n_devices = 0
    users = [None] * conf['n_users']
    for i in range(conf['n_users']):
        users[i] = User(address_book, net, conf)
        n_devices += users[i].n_devices

    #     print("User #{} has {} devices: {}".format(
    #         i+1, users[i].n_devices,
    #         [d.addr for d in users[i].devices]))
    print("initialized {} users and {} devices".format(conf['n_users'], n_devices))
    return users, net, address_book

def perform_round(conf, users, address_book_df=None, devices_view_df=None,
                  current_round=None, sleep=False):
    t_start = datetime.now()
    #print("Round #{}/{} at {}".format(t+1, conf['n_rounds'], datetime.now().time()))
    for i in range(conf['n_users']):
        users[i].act()
        
        if devices_view_df is not None:
            for d in users[i].devices:
                devices_view_df = devices_view_df.append(
                    d.peers_view.snapshot({'round': t}))
    if address_book_df is not None:
        address_book_df = address_book_df.append(
            address_book.snapshot({'round': t}))
    
    if sleep:
        elapsed = datetime.now() - t_start
        if elapsed > PERIOD:
            print("We are running overtime (took {})! You should increase conf['period'].".format(elapsed))
        elif t != conf['n_rounds'] - 1:
            #print("Sleeping", conf['period'] - elapsed)
            time.sleep((conf['period'] - elapsed).total_seconds())
    
    if current_round is None:
        current_round = 0
    else:
        current_round += 1
        
    return current_round, address_book_df, devices_view_df

def get_online_device(users):
    user = random.choice(users)
    while not user.has_online_devices():
        user = random.choice(users)
    return user.get_online_device()
def get_devices_pair(users):
    d1, d2 = get_online_device(users), get_online_device(users)
    while d1.owner == d2.owner:
        d2 = get_online_device(users)
    return d1, d2

# d1_status = address_book_df[address_book_df['addr'] == d1.addr]
# d2_status = address_book_df[address_book_df['addr'] == d2.addr]

def make_route(d1, d2):
    layers = [None] * 3
    layers[0] = d1.pick_devices(1)[0]
    layers[1:] = d2.pick_devices(2)
    return layers

In [62]:
conf = {
    'gossip_size': 20,
    'layer_threshold': 0.001,
    'minimum_node_availability': 0,
    'n_layers': 3,  # must be odd
    'n_rounds': 20,
    'n_users': 20,
    'period': timedelta(seconds=1),
}
address_book_df = pd.DataFrame()
devices_view_df = pd.DataFrame()
users, net, address_book = init_experiment(conf)

current_round, address_book_df, devices_view_df = perform_round(
    conf, users, address_book_df, devices_view_df)

initialized 20 users and 115 devices


In [63]:
d1, d2 = get_devices_pair(users)

route = [None] * conf['n_layers']
route[:conf['n_layers']//2] = d1.build_route(role='sender')
route[conf['n_layers']//2:] = d1.build_route(role='receiver')
route

[build_layer] Starting with view of size 20 and layer_threshold of 0.001.
[build_layer it. 0] Selected device having proba of 0.72. Now view has size 19 and p=0.2842
[build_layer it. 1] Selected device having proba of 0.96. Now view has size 18 and p=0.0107
[build_layer it. 2] Selected device having proba of 0.71. Now view has size 17 and p=0.0031
[build_layer it. 3] Selected device having proba of 0.81. Now view has size 16 and p=0.0006

[build_layer] Starting with view of size 20 and layer_threshold of 0.001.
[build_layer it. 0] Selected device having proba of 0.79. Now view has size 19 and p=0.2135
[build_layer it. 1] Selected device having proba of 0.43. Now view has size 18 and p=0.1212
[build_layer it. 2] Selected device having proba of 0.97. Now view has size 17 and p=0.0034
[build_layer it. 3] Selected device having proba of 0.34. Now view has size 16 and p=0.0022
[build_layer it. 4] Selected device having proba of 0.96. Now view has size 15 and p=0.0001

[build_layer] Starting

[                                    addr         p                          t  \
 2018-04-21 18:33:22.166582  gkf8u0ro637z  0.715835 2018-04-21 18:33:22.166582   
 2018-04-21 18:33:21.770321  ak5pxs2uox67  0.962414 2018-04-21 18:33:21.770321   
 2018-04-21 18:33:22.052043  g7sjvx1oaaer  0.713270 2018-04-21 18:33:22.052043   
 2018-04-21 18:33:21.880019  2wgoc8218bl6  0.810721 2018-04-21 18:33:21.880019   
 
                               type  
 2018-04-21 18:33:22.166582  mobile  
 2018-04-21 18:33:21.770321  server  
 2018-04-21 18:33:22.052043  mobile  
 2018-04-21 18:33:21.880019  mobile  ,
                                     addr         p                          t  \
 2018-04-21 18:33:22.071714  6g0o24zefhon  0.786523 2018-04-21 18:33:22.071714   
 2018-04-21 18:33:21.838989  5mi9g2vjqp1j  0.432126 2018-04-21 18:33:21.838989   
 2018-04-21 18:33:22.058241  6pv3btxnitm0  0.972266 2018-04-21 18:33:22.058241   
 2018-04-21 18:33:22.002585  u1qjea6blaqt  0.335290 2018-04-21 18:33:

In [64]:
current_round, address_book_df, devices_view_df = perform_round(
    conf, users, address_book_df, devices_view_df)

# Check if the route is still available (not the two ends)
opened = True
for l_id, layer in enumerate(route):
    are_devices_online = [net.get_device(addr).is_online for addr in layer['addr']]
    print("{}/{} online devices in layer {}.".format(
        sum(are_devices_online), len(layer), l_id + 1))
    opened = opened and any(are_devices_online)

    if not opened:
        break
if opened:
    print("The route is still opened!")
else:
    print("The route is closed.")

4/4 online devices in layer 1.
5/5 online devices in layer 2.
5/5 online devices in layer 3.
The route is still opened!


## Routes statistics

In [ ]:
N_EXPERIMENTS = 100
N_ROUTES_PER_EXPERIMENT = 10
N_ROUNDS_SINCE_ROUTE = 10
for n_layers in [3, 5, 7]:
    for layer_threshold in [0.1, 0.01, 0.001, 0.0001]:
        for _ in range(N_EXPERIMENTS):
            n_users = n_layers * 5
            conf = {
                'gossip_size': 20,
                'layer_threshold': 0.001,
                'minimum_node_availability': 0,
                'n_layers': 3,  # must be odd
                'n_rounds': N_ROUNDS_SINCE_ROUTE + 1,
                'n_users': n_users,
                'period': timedelta(seconds=1),
            }

# Test of PeersView

In [116]:
addresses = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
N = 20
EXPIRATION_PERIOD = timedelta(seconds=1)
print("Expiration period:", EXPIRATION_PERIOD)

address_book = PeersView(expiration_period=EXPIRATION_PERIOD)

for _ in range(N):
    connected = random.sample(addresses, k=random.randint(1, int(2 * len(addresses) / 3)))
    print("Time {}: connected = {}".format(datetime.now().time(), connected))
    
    for d in connected:
        address_book.put(datetime.now(), d, 0.8)
        time.sleep(0.1)
    
    print("Address book:")
    print(address_book.view)
    print()

Expiration period: 0:00:01
Time 00:01:59.105652: connected = ['g']


TypeError: put() missing 1 required positional argument: 'p'